In [135]:
import mysql.connector
from mysql.connector import Error 
import json
import pandas as pd
import datetime

# Necessary Methods(DO NOT CHANGE)

In [2]:
def get_connection():
    connection = mysql.connector.connect()
    return connection

In [3]:
def close_connection(connection):
    if connection:
        connection.close()

In [4]:
def query_database(command):
    try:
        connection = get_connection()
        db_Info = connection.get_server_info()
        print("Connected to MySQL Server version ", db_Info)
        cursor = connection.cursor()
        cursor.execute("select database();")
        record = cursor.fetchone()
        print("You're connected to database: ", record)    
        cursor.execute(command)
        record = cursor.fetchall()
        close_connection(connection)
        return record
    except (Exception, mysql.connector.Error) as error:
        print("Error while getting data", error)

In [5]:
def insert_database(command,data):
    connection = get_connection()
    db_Info = connection.get_server_info()
    print("Connected to MySQL Server version ", db_Info)
    cursor = connection.cursor()
    cursor.execute("select database();")
    record = cursor.fetchone()
    print("You're connected to database: ", record)  
    errors=[]
    for row in data:
        try:
            cursor.execute(command,row)
            connection.commit()
        except (Exception, mysql.connector.Error) as error:
            print("Error while inputting data",row,"\n", error,"\n")
            errors.append(row)
    close_connection(connection)
    return errors

In [6]:
def dateConvertor(year,month,day):
    return f"{year}-{month}-{day} 00:00:00"

In [7]:
avalible_regions = set(['CN','HK/TW','ID', 'JP', 'KR', 'MY/SG', 'OCE', 'PH', 'SA', 'TH', 'VN','SEA'])
TournamentRating = set(['S','A','B','C'])

# Insert New Tournament

## Insert Manually

The Following is the format you want to use if inserting manually

tournament_name= "Test" # tournament name<br>
tournament_region = "SA" # region tournament was (must be valid region)<br>
tournament_date_start= dateConvertor(2022,12,9) #only change the number in the parenthesis it goes year,month,day <br>
tournament_rating = "A"  # the rating of the tournament<br>
[optional] tournament_outsideID= "info:1811" #optional: add a outside Id if you want

In [8]:
tournament_name= "Test3"
tournament_region = "SA"
tournament_date_start= dateConvertor(2022,12,9)
tournament_date_end= dateConvertor(2022,12,11)
tournament_rating = "A"
tournament_outsideID=None
stmt = "INSERT INTO `ValoDex`.`Tournament` (`Tournament_Name`,`Region`,`Tournament_Rating_Type`,`DateStart`,`DateEnd`,`outsideID`) VALUES (%s,%s,%s,%s,%s,%s)"
insert_database(stmt,[(tournament_name,tournament_region,tournament_rating,tournament_date_start,tournament_date_end,tournament_outsideID)])

Connected to MySQL Server version  8.0.28
You're connected to database:  ('valodex',)


[]

## Insert through CSV


When inserting through csv make sure that the each line of the file is in the following format

Name,Region,Rating,StartDate,EndDate, [optional] outsideID

StartDate and EndDate have special Format as day,month,year the inbetween between the dates can be either - or /

Examples:<br>
Test,SA,A,9-12-2022,9-15-2022,<br>
Test2,TH,A,9/12/2022,9/15/2022,11

In [9]:
filename="tournament.csv"
tournament=[]
with open(filename,mode="r",encoding="utf-8") as f:
    for line in f:
        row=[]
        temp=line.replace("\n","").split(',')
        row.append(temp[0])
        row.append(temp[1])
        row.append(temp[2])
        datetemp=str(temp[3]).replace('/','-').split('-')
        row.append(dateConvertor(year=datetemp[2],month=datetemp[0],day=datetemp[1]))
        datetemp=str(temp[4]).replace('/','-').split('-')
        row.append(dateConvertor(year=datetemp[2],month=datetemp[0],day=datetemp[1]))
        if temp[5]!="":
            row.append(temp[5])
        else:
            row.append(None)
        tournament.append(row.copy())

In [10]:
print("Name,Region,Rating,StartDate,EndDate")
for row in tournament:
    print(row)

Name,Region,Rating,StartDate,EndDate
['Test', 'SA', 'A', '2022-9-12 00:00:00', '2022-9-15 00:00:00', None]
['Test2', 'SA', 'A', '2022-9-12 00:00:00', '2022-9-15 00:00:00', '11']


In [11]:
stmt = "INSERT INTO `ValoDex`.`Tournament` (`Tournament_Name`,`Region`,`Tournament_Rating_Type`,`DateStart`,`DateEnd`,`OutsideID`) VALUES (%s,%s,%s,%s,%s,%s)"
errors = insert_database(stmt,tournament)

Connected to MySQL Server version  8.0.28
You're connected to database:  ('valodex',)


## Update the Rating of a Tournament

The Following is the format you want to use if inserting manually

Tournament_rating='S'  #The rating you want to change to<BR>
Tournamnet_name='Test' #The name of the tournament<BR>

In [40]:
Tournament_rating='S'
Tournamnet_name='Test2'

stmt="update `ValoDex`.`Tournament` set `Tournament_Rating_Type`=%s where `Tournament_name`=%s "
insert_database(stmt,[(Tournament_rating,Tournamnet_name)])

Connected to MySQL Server version  8.0.28
You're connected to database:  ('valodex',)


[]

# Insert New Team

## Manual input

The following is the format you have to follow to do manual input:

teamname= "Bleed" #name<br>
teamabr="BLD"  #abbreviation<br>
region="MY/SG" #region ( must be a valid one)<br>
fa=0 # 0 mean not fa team | 1 mean fa team

In [12]:
teamname="Test"
teamabr="TT"
region="CN"
fa=0

stmt = "INSERT INTO `ValoDex`.`Team` (`Name`,`Abbreviation`,`Region`,`FA`) VALUES (%s,%s,%s,%s)"
insert_database(stmt,[(teamname,teamabr,region,fa)])

Connected to MySQL Server version  8.0.28
You're connected to database:  ('valodex',)


[]

## Input through CSV

When inserting through csv make sure that the each line of the file is in the following format

team_name,team_ abreviation,region, and fa status

example:<br> 
Bleed,BLD,MY/SG,0<br>
Boom,BME,ID,0<br>
Made in Thailand,MITH,TH,0

In [13]:
filename="team.csv"
team=[]
with open(filename,mode="r",encoding="utf-8") as f:
    for line in f:
        line=str(line).upper()
        temp=line.replace("\n","").split(',')
        temp[3]=int(temp[3])
        team.append(temp)
        
        

In [14]:
print("Name,Abreviation,Region,FA")
for row in team:
    print(row)   

Name,Abreviation,Region,FA
['BLEED', 'BLD', 'MY/SG', 0]
['BOOM', 'BME', 'ID', 0]
['MADE IN THAILAND', 'MITH', 'TH', 0]


In [15]:
stmt = "INSERT INTO `ValoDex`.`Team` (`Name`,`Abbreviation`,`Region`,`FA`) VALUES (%s,%s,%s,%s)"
insert_database(stmt,team)

Connected to MySQL Server version  8.0.28
You're connected to database:  ('valodex',)


[]

# Update ELO SCALE

The following is the format you have to follow to do manual input:

region="MY/SG" #region ( must be a valid one)<br>
eloscale=30 # the percentage by number (i.e. 30 = 30%,20=20%)

In [16]:
region='SA'
eloscale=30
stmt = "Update `ValoDex`.`Region` Set `EloScale`=%s where `Name`=%s"
insert_database(stmt,[(eloscale,region)])   

Connected to MySQL Server version  8.0.28
You're connected to database:  ('valodex',)


[]

# Update Tournament Rating

The following is the format you have to follow to do manual input:

TournamentType="A" #The tournament rating<BR>
Placement=1 #The placement you want to change<BR>
PlacementElo=50 #The elo you want to set it to

In [17]:
TournamentType="A"
Placement=1
PlacementElo=50
stmt = "Update `ValoDex`.`Rating_Placement` Set `Elo`=%s where `Tournament_Rating_Type`=%s and `Placement`=%s"
insert_database(stmt,[(PlacementElo,TournamentType,Placement)])

Connected to MySQL Server version  8.0.28
You're connected to database:  ('valodex',)


[]

# Tournament has Team

Tournament info

In [25]:
tournament_name= "Test2"
tournament_date_start= dateConvertor(2022,9,12)
stmt = f"Select `Tournament_Rating_Type` FROM `ValoDex`.`tournament` WHERE `Tournament_Name`='{tournament_name}' and `DateStart`= '{tournament_date_start}'"
tournament_rating=query_database(stmt)[0][0]

Connected to MySQL Server version  8.0.28
You're connected to database:  ('valodex',)


Tournament Placements

In [48]:
filename="book1.csv"
team_placement=[]
with open(filename,mode="r",encoding="utf-8") as f:
    for x in f:
        x=x.replace("\n","")
        x=x.split(",")
        teamname=x[1]
        region=x[2]
        placement=x[0]
        elo_placement=(int(placement)+1)//2
        team_placement.append((tournament_name,tournament_date_start,tournament_rating,teamname,region,placement,elo_placement))
team_placement

[('Test', '2022-9-12 00:00:00', 'A', 'BLEED', 'MY/SG', '1', 1),
 ('Test', '2022-9-12 00:00:00', 'A', 'BOOM', 'ID', '2', 1),
 ('Test', '2022-9-12 00:00:00', 'A', 'TEST', 'CN', '3', 2)]

In [27]:
stmt=""
stmt = "INSERT INTO `ValoDex`.`Tournament_has_Team` (`Tournament_Name`,`"
stmt +="Tournament_DateStart`,`Tournament_Rating_Type`,`Team_Name`,`Team_Region`,"
stmt+="`Placement`,`Elo_Placement`) VALUES (%s,%s,%s,%s,%s,%s,%s)"
insert_database(stmt,team_placement)

Connected to MySQL Server version  8.0.28
You're connected to database:  ('valodex',)


[]

## Update Extra Elo Scale

The following is the format you have to follow to do manual input:<BR>
    
tournament_name= "Test" # the name of the tournament you want to change<BR>
team_name="BOOM" # The name of the team this change is for<BR>

Extra_ELO=20 # ther percentage of elo you need to change<BR>
Reason="Surrender" #the reason for the change

In [33]:
tournament_name= "Test"
team_name="BOOM"

Extra_ELO=20
Reason="Surrender"
data= [(Extra_ELO,Reason,tournament_name,team_name)]
print(data)

[(20, 'Surrender', 'Test', 'BOOM')]


In [34]:
stmt=""
stmt+="update `ValoDex`.`Tournament_has_Team`"
stmt+="set `Extra_Elo_Scale` = %s,`Reason_For_Extra` = %s "
stmt+="where `Tournament_has_Team`.`Tournament_Name`=%s "
stmt+="and `Tournament_has_Team`.`Team_Name` = %s"
insert_database(stmt,data)

Connected to MySQL Server version  8.0.28
You're connected to database:  ('valodex',)


[]

# Calculations

select `Tournament_has_Team`.`Team_Name`,`Tournament_has_Team`.`Team_Region`,
Count(`Tournament_has_Team`.`Team_Name`) as `Tourney Count`,Sum(`Rating_Placement`.`Elo`*(100-(`Region`.`EloScale`+`Tournament_has_team`.`Extra_Elo_scale`))/100)+1000 as `Elo`
from `Tournament_has_team`,`rating_placement`,`tournament`,`Region`
where `Tournament_has_team`.`Tournament_Rating_Type`=`rating_placement`.`Tournament_Rating_Type`
and `Tournament_has_team`.`Elo_Placement` = `rating_placement`.`placement` 
and `Tournament_has_Team`.`Tournament_Name`= `Tournament`.`Tournament_Name`
and `Tournament`.`Region` = `Region`.`Name`
Group By `Tournament_has_team`.`Team_Name`,`Tournament_has_Team`.`Team_Region`
Order by `Elo` DESC


In [35]:
stmt = ""
stmt+="select `Tournament_has_Team`.`Team_Name`,`Tournament_has_Team`.`Team_Region`,"
stmt+="Count(`Tournament_has_Team`.`Team_Name`) as Count,Sum(`Rating_Placement`.`Elo`*(100-(`Region`.`EloScale`+`Tournament_has_team`.`Extra_Elo_scale`))/100)+1000 as `Elo`"
stmt+="from `Tournament_has_team`,`rating_placement`,`tournament`,`Region`"
stmt+="where `Tournament_has_team`.`Tournament_Rating_Type`=`rating_placement`.`Tournament_Rating_Type`"
stmt+="and `Tournament_has_team`.`Elo_Placement` = `rating_placement`.`placement`"
stmt+="and `Tournament_has_Team`.`Tournament_Name`= `Tournament`.`Tournament_Name`"
stmt+="and `Tournament`.`Region` = `Region`.`Name`"
stmt+="Group By `Tournament_has_team`.`Team_Name`,`Tournament_has_Team`.`Team_Region`"
stmt+="Order by `Elo` DESC,`Count`ASC"

store= query_database(stmt)
with open("test.csv","w") as f:
    f.write(",Team,Region,Tourney Count,Elo\n")
    for i,row in enumerate(store):
        f.write(str(i+1))
        for data in row:
            f.write(",")
            f.write(str(data))
        f.write("\n")

Connected to MySQL Server version  8.0.28
You're connected to database:  ('valodex',)


# Tournament Results

select `Tournament_has_Team`.`Tournament_Name`,`Tournament_has_Team`.`Tournament_DateStart`,`Tournament`.`Region` as `TournamentRegion`,
`Tournament_has_Team`.`Tournament_Rating_Type`,`Tournament_has_Team`.`Team_Name`,`Tournament_has_Team`.`Team_Region`,
`Tournament_has_team`.`Placement`,`Tournament_has_team`.`Elo_Placement`,`Rating_Placement`.`Elo`,`Region`.`EloScale`,`Tournament_has_team`.`Extra_Elo_Scale`,
`Rating_Placement`.`Elo`*(100-(`Region`.`EloScale`+`Tournament_has_team`.`Extra_Elo_scale`))/100 as `Elo_Gained`
from `Tournament_has_team`,`rating_placement`,`tournament`,`Region`
where `Tournament_has_team`.`Tournament_Rating_Type`=`rating_placement`.`Tournament_Rating_Type`
and `Tournament_has_team`.`Elo_Placement` = `rating_placement`.`placement` 
and `Tournament_has_Team`.`Tournament_Name`= `Tournament`.`Tournament_Name`
and `Tournament`.`Region` = `Region`.`Name`
Group by `Tournament_has_Team`.`Team_Name`,`Tournament_has_Team`.`Team_Region`,`Tournament_has_Team`.`Tournament_Name`,`Tournament_has_Team`.`Tournament_DateStart`,`TournamentRegion`,
`Tournament_has_Team`.`Tournament_Rating_Type`,`Tournament_has_Team`.`Team_Name`,`Tournament_has_Team`.`Team_Region`,
`Tournament_has_team`.`Placement`,`Tournament_has_team`.`Elo_Placement`,`Rating_Placement`.`Elo`,`Region`.`EloScale`,`Elo_Gained`


In [24]:
stmt =""
stmt+="select `Tournament_has_Team`.`Tournament_Name`,`Tournament_has_Team`.`Tournament_DateStart`,`Tournament`.`Region` as `TournamentRegion`,"
stmt+="`Tournament_has_Team`.`Tournament_Rating_Type`,`Tournament_has_Team`.`Team_Name`,`Tournament_has_Team`.`Team_Region`,"
stmt+="`Tournament_has_team`.`Placement`,`Tournament_has_team`.`Elo_Placement`,`Rating_Placement`.`Elo`,`Region`.`EloScale`,`Tournament_has_team`.`Extra_Elo_Scale`,"
stmt+="`Rating_Placement`.`Elo`*(100-(`Region`.`EloScale`+`Tournament_has_team`.`Extra_Elo_scale`))/100 as `Elo_Gained`"
stmt+="from `Tournament_has_team`,`rating_placement`,`tournament`,`Region`"
stmt+="where `Tournament_has_team`.`Tournament_Rating_Type`=`rating_placement`.`Tournament_Rating_Type`"
stmt+="and `Tournament_has_team`.`Elo_Placement` = `rating_placement`.`placement`" 
stmt+="and `Tournament_has_Team`.`Tournament_Name`= `Tournament`.`Tournament_Name`"
stmt+="and `Tournament`.`Region` = `Region`.`Name`"
stmt+="Group by `Tournament_has_Team`.`Team_Name`,`Tournament_has_Team`.`Team_Region`,`Tournament_has_Team`.`Tournament_Name`,`Tournament_has_Team`.`Tournament_DateStart`,`TournamentRegion`,"
stmt+="`Tournament_has_Team`.`Tournament_Rating_Type`,`Tournament_has_Team`.`Team_Name`,`Tournament_has_Team`.`Team_Region`,"
stmt+="`Tournament_has_team`.`Placement`,`Tournament_has_team`.`Elo_Placement`,`Rating_Placement`.`Elo`,`Region`.`EloScale`,`Elo_Gained`"
store=query_database(stmt)
for row in store:
    print(row)

Connected to MySQL Server version  8.0.28
You're connected to database:  ('valodex',)
('Test', datetime.datetime(2022, 9, 12, 0, 0), 'SA', 'A', 'BLEED', 'MY/SG', 1, 1, 50, 30, 0, Decimal('35.0000'))
('Test', datetime.datetime(2022, 9, 12, 0, 0), 'SA', 'A', 'BOOM', 'ID', 2, 1, 50, 30, 20, Decimal('25.0000'))
('Test', datetime.datetime(2022, 9, 12, 0, 0), 'SA', 'A', 'TEST', 'CN', 3, 2, 100, 30, 0, Decimal('70.0000'))


# Download Tables

Run everything here when you want to save the data

In [124]:
tables={}
tables["region"]=pd.DataFrame(query_database("select * from `ValoDex`.`Region`"),
                              columns=["Name", "EloScale"]).to_csv(index=False)
tables["tournament"]=pd.DataFrame(query_database("select * from `ValoDex`.`Tournament`"),
                                    columns=["ValoDexID", "outsideID", "Tournament_Name", "Region", "DateStart", "DateEnd", "Tournament_Rating_Type"]).to_csv(index=False)
tables["tournament_rating"]=pd.DataFrame(query_database("select * from `ValoDex`.`tournament_rating`"),columns=["Type"]).to_csv(index=False)
tables["team"]=pd.DataFrame(query_database("select * from `ValoDex`.`Team`"),columns=["TeamID", "Name", "Abbreviation", "Region", "FA"]).to_csv( index=False)
tables["tournamnet_has_team"]=pd.DataFrame(query_database("select * from `ValoDex`.`Tournament_has_team`"),
                                          columns=["Tournament_Name", "Tournament_DateStart", "Tournament_Rating_Type", "Team_Name", "Team_Region", "Placement", "Elo_placement", "Extra_Elo_Scale", "Reason_For_Extra"]).to_csv(index=False)
tables["rating_placement"]=pd.DataFrame(query_database("select * from `ValoDex`.`Rating_Placement`"),columns=["Placement", "Tournament_Rating_Type", "Elo"]).to_csv(index=False)

Connected to MySQL Server version  8.0.28
You're connected to database:  ('valodex',)
Connected to MySQL Server version  8.0.28
You're connected to database:  ('valodex',)
Connected to MySQL Server version  8.0.28
You're connected to database:  ('valodex',)
Connected to MySQL Server version  8.0.28
You're connected to database:  ('valodex',)
Connected to MySQL Server version  8.0.28
You're connected to database:  ('valodex',)
Connected to MySQL Server version  8.0.28
You're connected to database:  ('valodex',)


In [128]:
for row in tables:
    print(f"{row}\n{tables[row]}")

region
Name,EloScale
CN,0
EA,0
HK/TW,0
ID,0
JP,0
KR,0
MY/SG,0
OCE,0
PH,0
SA,30
SEA,0
TH,0
VN,0

tournament
ValoDexID,outsideID,Tournament_Name,Region,DateStart,DateEnd,Tournament_Rating_Type
2,,Test,SA,2022-09-12,2022-09-15,S
3,11,Test2,SA,2022-09-12,2022-09-15,S
1,,Test3,SA,2022-12-09,2022-12-11,A

tournament_rating
Type
A
B
C
S

team
TeamID,Name,Abbreviation,Region,FA
2,BLEED,BLD,MY/SG,0
3,BOOM,BME,ID,0
4,MADE IN THAILAND,MITH,TH,0
1,Test,TT,CN,0

tournamnet_has_team
Tournament_Name,Tournament_DateStart,Tournament_Rating_Type,Team_Name,Team_Region,Placement,Elo_placement,Extra_Elo_Scale,Reason_For_Extra
Test,2022-09-12,S,BLEED,MY/SG,1,1,0,
Test,2022-09-12,S,BOOM,ID,2,1,20,Surrender
Test,2022-09-12,S,TEST,CN,3,2,0,
Test2,2022-09-12,S,BLEED,MY/SG,1,1,0,
Test2,2022-09-12,S,BOOM,ID,2,1,0,
Test2,2022-09-12,S,TEST,CN,3,2,0,

rating_placement
Placement,Tournament_Rating_Type,Elo
1,A,50
1,S,100
2,A,100
2,S,100
4,A,100
4,S,100
8,A,100
8,S,100



In [144]:
filename="download_SQL.txt"
with open(filename,mode="w",encoding="utf-8") as writefile:
    writefile.write("Time downloaded: ")
    writefile.write(str(datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")))
    writefile.write("\n")
    for row in tables:
        writefile.write(f"{row}\n{tables[row]}\n")